In [1]:
import numpy as np
!pip install pytorch-lightning

In [2]:
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.nn.functional as F

C:\Users\ogurc\AppData\Local\Programs\Python\Python310\lib\site-packages\torchaudio\backend\utils.py:74: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")


In [3]:
import os
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [46]:
activation_fun = nn.ReLU()
layer_sizes = [1, 8, 8, 8, 1]


class SamenessModule(nn.Module):
    def __init__(self):
        super().__init__()

        self.run_counter = 0

        layers_list = []
        for i in range(len(layer_sizes) - 1):
            layers_list.append(nn.Linear(layer_sizes[i], layer_sizes[i + 1]))
            if i != len(layer_sizes) - 2:
                layers_list.append(activation_fun)

        self.l1 = nn.Sequential(*layers_list)
        # self.layers = layers_list
        print(self)

        for layer in self.l1:
            if isinstance(layer, nn.Linear):
                layer.weight = nn.Parameter(layer.weight.double())
                layer.bias = nn.Parameter(layer.bias.double())

    def forward(self, x):
        self.run_counter += 1
        return self.l1(x.double())  # <--- Вылетает ошибка тут


# class Decoder(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.l1 = nn.Sequential(nn.Linear(3, 64), nn.ReLU(), nn.Linear(64, 28 * 28))
#
#     def forward(self, x):
#         return self.l1(x)


class SamenessAutoEncoder(pl.LightningModule):
    def __init__(self, encoder: SamenessModule):
        super().__init__()
        self.encoder = encoder

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        # print("\n" + "-" * 20)
        # print(f"batch: {batch}")
        # print("-" * 20)
        x, y = batch[0][0]
        print(f"batch:{'-' * 10}\n{x}\n{y}\n{'-' * 20}")
        x_hat = self.encoder(x)
        loss = F.mse_loss(x_hat, x)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

In [47]:
from torch.utils.data import Dataset
import torch
import os
import torch.random
import pandas as pd

In [48]:
# from notebooks.data.SimpleRandomDataset import SimpleRandomDataset

In [49]:
class SimpleRandomDataset(Dataset):
    """Simplest random dataset"""

    def __init__(self, n, left_bound=-5, right_bound=5, transform=None):
        """

        :param left_bound:
        :param right_bound:
        """
        self.size = n
        self.transform = transform
        random_array = np.random.rand(n) * (right_bound - left_bound) + left_bound
        random_array = random_array.reshape((len(random_array), 1))
        self.data = np.array([random_array, random_array])
        # self.data = pd.DataFrame({
        #     'X': random_array,
        #     'y': random_array,
        # })

    def __len__(self):
        return self.size

    def __getitem__(self, idx):
        print(idx)
        # img_name = os.path.join(self.root_dir,
        #                         self.landmarks_frame.iloc[idx, 0])
        # image = io.imread(img_name)
        # landmarks = self.landmarks_frame.iloc[idx, 1:]
        # landmarks = np.array([landmarks])
        # landmarks = landmarks.astype('float').reshape(-1, 2)
        # sample = {'image': image, 'landmarks': landmarks}
        #
        # if self.transform:
        #     sample = self.transform(sample)
        #
        # return sample
        # pass
        if torch.is_tensor(idx):
            idx = idx.tolist()
        # if isinstance(idx, int):
        #     idx = [idx]

        # X = self.data[idx]
        # y = self.data[idx]
        # sample = {"X": X, "y": y}

        # sample = self.data.iloc[idx].to_numpy()
        sample = self.data[:, idx]
        # sample = np.array([X, y])

        # FIXME:
        # if idx is int:
        #     sample = self.data.iloc[idx].to_list()
        #     sample = [[sample[0]], [sample[1]]]
        # else:
        # sample = self.data.iloc[idx].to_numpy().T

        # print("\n" + "-" * 20)
        # print(f"sample: {sample}")
        # print("-" * 20)
        if self.transform:
            sample = self.transform(sample)
        # print(f"sample: {sample}")
        # print("-" * 20)
        return sample

In [51]:
dataset = SimpleRandomDataset(100, transform=transforms.ToTensor())
train_loader = DataLoader(dataset, batch_size=10)

# model
autoencoder = SamenessAutoEncoder(SamenessModule())

# train model
trainer = pl.Trainer(max_epochs=3)
# trainer.fit(model=autoencoder, train_dataloaders=train_loader)
trainer.fit(model=autoencoder, train_dataloaders=train_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3050 Ti Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type           | Params
-------------------------------------------
0 | encoder | SamenessModule | 169   
-------------------------------------------
169       Trainable params
0         Non-trainable params
169       Total params
0.001     Total estimated model params size (MB)
C:\Users\ogurc\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_lightning\loops\fit_loop.py:280: PossibleUserWarning

SamenessModule(
  (l1): Sequential(
    (0): Linear(in_features=1, out_features=8, bias=True)
    (1): ReLU()
    (2): Linear(in_features=8, out_features=8, bias=True)
    (3): ReLU()
    (4): Linear(in_features=8, out_features=8, bias=True)
    (5): ReLU()
    (6): Linear(in_features=8, out_features=1, bias=True)
  )
)
Epoch 0:   0%|          | 0/10 [00:00<?, ?it/s] 0
1
2
3
4
5
6
7
8
9
batch:----------
tensor([0.2073], device='cuda:0', dtype=torch.float64)
tensor([0.2073], device='cuda:0', dtype=torch.float64)
--------------------
Epoch 0:  10%|█         | 1/10 [00:00<00:00, 76.91it/s, v_num=106]10
11
12
13
14
15
16
17
18
19
batch:----------
tensor([-3.6697], device='cuda:0', dtype=torch.float64)
tensor([-3.6697], device='cuda:0', dtype=torch.float64)
--------------------
Epoch 0:  20%|██        | 2/10 [00:00<00:00, 83.31it/s, v_num=106]20
21
22
23
24
25
26
27
28
29
batch:----------
tensor([-4.7279], device='cuda:0', dtype=torch.float64)
tensor([-4.7279], device='cuda:0', dtype=torch.

`Trainer.fit` stopped: `max_epochs=3` reached.


Epoch 2: 100%|██████████| 10/10 [00:00<00:00, 83.34it/s, v_num=106]


In [24]:
from tqdm import tqdm

In [55]:
test_dataset = SimpleRandomDataset(100, transform=transforms.ToTensor())
test_loader = DataLoader(test_dataset, batch_size=10)

with torch.no_grad():
    # for batch in tqdm(test_loader):
    for batch in tqdm(test_loader):
        x, y = batch[0][0]
        reconstructed_x = autoencoder.encoder(x)
        print("Original x:", x)
        print("Reconstructed x:", reconstructed_x)



  0%|          | 0/10 [00:00<?, ?it/s]

0
1
2
3
4
5
6
7
8
9
Original x: tensor([4.2871], dtype=torch.float64)
Reconstructed x: tensor([0.1040], dtype=torch.float64)



 10%|█         | 1/10 [04:02<36:21, 242.43s/it]

10
11
12
13
14
15
16
17
18
19
Original x: tensor([2.1474], dtype=torch.float64)
Reconstructed x: tensor([0.0318], dtype=torch.float64)



 20%|██        | 2/10 [04:53<17:18, 129.86s/it]

...

50
51
52
53
54
55
56
57
58
59
Original x: tensor([2.3880], dtype=torch.float64)
Reconstructed x: tensor([0.0382], dtype=torch.float64)



 70%|███████   | 7/10 [07:27<01:48, 36.02s/it]

60
61
62
63
64
65
66
67
68
69
Original x: tensor([0.9512], dtype=torch.float64)
Reconstructed x: tensor([0.0331], dtype=torch.float64)



100%|██████████| 10/10 [07:29<00:00, 44.97s/it][A

70
71
72
73
74
75
76
77
78
79
Original x: tensor([-3.0999], dtype=torch.float64)
Reconstructed x: tensor([-0.1758], dtype=torch.float64)
80
81
82
83
84
85
86
87
88
89
Original x: tensor([1.9881], dtype=torch.float64)
Reconstructed x: tensor([0.0335], dtype=torch.float64)
90
91
92
93
94
95
96
97
98
99
Original x: tensor([0.5828], dtype=torch.float64)
Reconstructed x: tensor([0.0253], dtype=torch.float64)


In [32]:
autoencoder.encoder.run_counter

3

In [ ]:
n = 1000
s = 0
c = 0
with torch.no_grad():
    for val in test_dataset:
        c += 1
        s += autoencoder.encoder(val)
print(f"avarage diff: {s / c}")

In [ ]:
autoencoder.encoder.run_counter

In [ ]:
val = torch.tensor([[1], [2], [4], [5]])
autoencoder.encoder(val)

In [57]:
x = np.linspace(-10, 10, 1000)
print(x)
x = x.reshape(len(x), 1)
print(x)

[-10.          -9.97997998  -9.95995996  -9.93993994  -9.91991992
  -9.8998999   -9.87987988  -9.85985986  -9.83983984  -9.81981982
  -9.7997998   -9.77977978  -9.75975976  -9.73973974  -9.71971972
  -9.6996997   -9.67967968  -9.65965966  -9.63963964  -9.61961962
  -9.5995996   -9.57957958  -9.55955956  -9.53953954  -9.51951952
  -9.4994995   -9.47947948  -9.45945946  -9.43943944  -9.41941942
  -9.3993994   -9.37937938  -9.35935936  -9.33933934  -9.31931932
  -9.2992993   -9.27927928  -9.25925926  -9.23923924  -9.21921922
  -9.1991992   -9.17917918  -9.15915916  -9.13913914  -9.11911912
  -9.0990991   -9.07907908  -9.05905906  -9.03903904  -9.01901902
  -8.998999    -8.97897898  -8.95895896  -8.93893894  -8.91891892
  -8.8988989   -8.87887888  -8.85885886  -8.83883884  -8.81881882
  -8.7987988   -8.77877878  -8.75875876  -8.73873874  -8.71871872
  -8.6986987   -8.67867868  -8.65865866  -8.63863864  -8.61861862
  -8.5985986   -8.57857858  -8.55855856  -8.53853854  -8.51851852
  -8.49849